In [15]:
import numpy as np 
import pandas as pd
from datetime import datetime

In [16]:
data_1hour = pd.read_csv('AAPL_full_1hour_adjsplit.csv')
data_1hour.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1day = pd.read_csv('AAPL_full_1day_adjsplit.csv')
data_1day.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1hour_dropped = data_1hour.drop([0,1,2,3,4,5,6,7,8,9], axis =0)
data_1hour_dropped = pd.DataFrame(data_1hour_dropped)
data_1hour_dropped['Date'] = pd.to_datetime(data_1hour_dropped['Date'], format="%Y-%m-%d %H:%M:%S")
data_1hour_dropped["Hourly Log Return"] = np.log(data_1hour_dropped['Close']).diff()

In [17]:
def calculate_intraday_volatility(dataframe):

    if 'Date' in dataframe.columns:
        dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d %H:%M:%S")

    date_log_return_df = dataframe[['Date', 'Hourly Log Return']]

    date_log_return_df.set_index('Date', inplace=True)
   
    intraday_volatility = date_log_return_df.resample('D').apply(lambda x: (x ** 2).sum(axis=0) / x.notna().sum(axis=0)).apply(lambda x: x ** 0.5)

    intraday_volatility.reset_index(inplace=True)

    intraday_volatility.columns = ['Date', 'Intraday_Volatility']

    return intraday_volatility

result_df = calculate_intraday_volatility(data_1hour_dropped)


/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46292/1216246990.py:10: RuntimeWarning: invalid value encountered in double_scalars
  intraday_volatility = date_log_return_df.resample('D').apply(lambda x: (x ** 2).sum(axis=0) / x.notna().sum(axis=0)).apply(lambda x: x ** 0.5)


In [18]:
def remove_zero_volatility(dataframe):

    filtered_dataframe = dataframe[dataframe['Intraday_Volatility'] != 0]

    return filtered_dataframe

newresults = remove_zero_volatility(result_df)


In [19]:
def find_dates_not_in_both(series1, series2):

    dates_not_in_series1 = series1[~series1.isin(series2)]
    dates_not_in_series2 = series2[~series2.isin(series1)]

series1 = data_1day["Date"]
series2 = newresults["Date"]

find_dates_not_in_both(series1, series2)

In [20]:
newresults2 = newresults.drop(newresults[newresults['Date'] == "2023-09-19"].index)

In [21]:
data_1day['Date'] = pd.to_datetime(data_1day['Date'])
newresults2['Date'] = pd.to_datetime(newresults2['Date'])
data_1day.set_index('Date', inplace=True)
newresults2.set_index('Date', inplace=True)

In [22]:
data_1day["Daily Volatility"] = newresults2["Intraday_Volatility"]
data_1day['Range'] = data_1day['High'] - data_1day['Low']
data_1day['Return'] = np.log(data_1day['Close']).diff()
data_1day['Return_Squared'] = np.square(data_1day['Return'])
data_1day = data_1day.dropna()

In [23]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [24]:
import yfinance as yf
import pandas_ta as ta
data = yf.download(tickers = 'AAPL', start = '2005-01-05', end = '2024-01-20')
data['EMAF'] = ta.ema(data.Close, length = 20)
data_1day["EMAF"] = data["EMAF"]
data = data_1day.dropna()
daily_volatility = data["Daily Volatility"]

[*********************100%%**********************]  1 of 1 completed


In [25]:
target = [1 if daily_volatility[i] > daily_volatility[i - 1] else -1 for i in range(1, len(daily_volatility))]
target = [0] + target

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46292/4277351889.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = [1 if daily_volatility[i] > daily_volatility[i - 1] else -1 for i in range(1, len(daily_volatility))]


In [26]:
df = pd.DataFrame({'daily_volatility': daily_volatility})
df['target'] = np.where(df['daily_volatility'] > df['daily_volatility'].shift(1), 1, 0)
df['target'].fillna(0, inplace=True)
data['target'] = df['target']
data_full = data.dropna()


/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46292/1836751241.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = df['target']


In [27]:
data_full = data_full[["Return_Squared","Return", "Range", "Volume",  "EMAF",  "Daily Volatility", "target"]]

In [28]:
data_full

,Return_Squared,Return,Range,Volume,EMAF,Daily Volatility,target
Date,,,,,,,
2005-02-02,0.000714,0.026725,0.0397,1024582888,1.262946,0.007372,0
2005-02-03,0.000535,-0.023120,0.0375,732986296,1.274996,0.004307,0
2005-02-04,0.000173,0.013155,0.0250,563683400,1.287649,0.003966,0
2005-02-07,0.000001,0.001207,0.0331,526396528,1.299268,0.005254,1
2005-02-08,0.000602,0.024526,0.0462,889647080,1.313113,0.005591,1
...,...,...,...,...,...,...,...
2024-01-12,0.000003,0.001777,1.5500,39664356,188.235134,0.001663,0
2024-01-16,0.000154,-0.012394,3.3260,63297662,187.796550,0.005651,1
2024-01-17,0.000027,-0.005187,2.6300,46287491,187.309259,0.002445,0


In [29]:
data_full.to_csv(r"Intraday Volatility Dataset.csv")